In [48]:
import pandas as pd
import psycopg2
from urllib.parse import quote_plus

## Conexao BD

In [49]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

usuario = 'postgres'
senha = quote_plus('@manaus')
host = 'localhost'
porta = '5432'
nome_do_banco = 'db_mrp'

# Configuração da conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{nome_do_banco}')

In [29]:
from sqlalchemy.types import Integer
# Configuração da conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{nome_do_banco}')

# Exportar o DataFrame para o banco de dados PostgreSQL
filtered_df.to_sql('table_plan_assy', con=engine, if_exists='replace', index=False)
print('Dados salvos com sucesso no banco de dados PostgreSQL.')

Dados salvos com sucesso no banco de dados PostgreSQL.


In [28]:
    
# Carregar a tabela BOM do banco de dados
df_bom = pd.read_sql("SELECT * FROM table_bom", engine)

# Filtrar as colunas relevantes
df_plan_assy = df_bom[['org', 'child_item', 'child_uit', 'qpa', 'assy', 'local', 'top_item']].copy()
    
# Criar a coluna ID combinando ORG e CHILD_ITEM
df_plan_assy['ID'] = df_plan_assy['org'].astype(str) + df_plan_assy['child_item'].astype(str)

# Inicializar a coluna 'infor' como vazia
df_plan_assy['infor'] = ''

# Preencher a coluna 'infor' com base nas condições
previous_value = None
for idx in range(len(df_plan_assy)):
    current_value = df_plan_assy.at[idx, 'child_item']
    if previous_value is not None and current_value != previous_value:
        df_plan_assy.at[idx, 'infor'] = 'Onhand'
        if idx + 1 < len(df_plan_assy):
            df_plan_assy.at[idx + 1, 'infor'] = 'Plano'
    else:
        df_plan_assy.at[idx, 'infor'] = 'Plano'
    previous_value = current_value

# Preencher 'Shortage' após 'Onhand'    
for idx in range(len(df_plan_assy) - 1):
    if df_plan_assy.at[idx, 'infor'] == 'Onhand':
        df_plan_assy.at[idx + 1, 'infor'] = 'Shortage'

# Adicionar 'Shortage' no final
df_plan_assy.at[len(df_plan_assy), 'infor'] = 'Shortage'
    
# Criar a coluna AnotherID com base nas condições
df_plan_assy['AnotherID'] = df_plan_assy.apply(
    lambda row: '' if row['infor'] == 'Onhand' else (
        str(row['child_item']) + str(row['assy']) 
        if row['infor'] == 'Plano' else 
        str(row['child_item']) + str(row['assy']) + 'Shortage'
    ),
    axis=1
)
    
# Aplicar o filtro na coluna 'child_uit' com os critérios ["M", "P", "R", "T"]
criterios = ["M", "P", "R", "T"]
filtered_df = df_plan_assy[df_plan_assy['child_uit'].isin(criterios)]

# Reorganizar as colunas na ordem desejada
ordered_columns = ['ID', 'AnotherID', 'org', 'child_item', 'child_uit', 'qpa', 'assy', 'local', 'top_item', 'infor']
filtered_df = filtered_df.reindex(columns=ordered_columns)
    
# Exibir o DataFrame resultante
filtered_df.head()

,ID,AnotherID,org,child_item,child_uit,qpa,assy,local,top_item,infor
0,NW1AAN30117605,AAN3011760532LR650BPSA.AWZSLJZ,NW1,AAN30117605,R,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Plano
2,NW1ACQ30597516,ACQ3059751632LR650BPSA.AWZSLJZShortage,NW1,ACQ30597516,T,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Shortage
4,NW1AAA31161802,AAA3116180232LR650BPSA.AWZSLJZShortage,NW1,AAA31161802,R,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Shortage
21,NW1EBT67983201,,NW1,EBT67983201,T,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Onhand
29,NW1MAZ67664402,,NW1,MAZ67664402,M,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Onhand


In [ ]:
import pandas as pd
import numpy as np

# Carregar a tabela BOM do banco de dados
df_bom = pd.read_sql("SELECT * FROM table_bom", engine)

# Filtrar as colunas relevantes
df_plan_assy = df_bom[['org', 'child_item', 'child_uit', 'qpa', 'assy', 'local', 'top_item']].copy()

# Criar a coluna ID combinando ORG e CHILD_ITEM
df_plan_assy['ID'] = df_plan_assy['org'].astype(str) + df_plan_assy['child_item'].astype(str)

# Inicializar a coluna 'infor' como 'Plano' por padrão
df_plan_assy['infor'] = 'Plano'

# Identificar mudanças na coluna 'child_item' e atualizar a coluna 'infor'
df_plan_assy['infor'] = np.where(
    df_plan_assy['child_item'].shift() != df_plan_assy['child_item'], 
    'Onhand', 
    df_plan_assy['infor']
)

# Adicionar linhas para "Shortage" após cada "Onhand"
shortage_rows = df_plan_assy[df_plan_assy['infor'] == 'Onhand'].copy()
shortage_rows['infor'] = 'Shortage'

# Concatenar as linhas de "Shortage" no DataFrame original
df_plan_assy = pd.concat([df_plan_assy, shortage_rows]).sort_index().reset_index(drop=True)

# Criar a coluna AnotherID com base nas condições
df_plan_assy['AnotherID'] = np.where(
    df_plan_assy['infor'] == 'Onhand', 
    '', 
    df_plan_assy['child_item'] + df_plan_assy['assy'] + 
    np.where(df_plan_assy['infor'] == 'Shortage', 'Shortage', '')
)

# Aplicar o filtro na coluna 'child_uit' com os critérios ["M", "P", "R", "T"]
criterios = ["M", "P", "R", "T"]
filtered_df = df_plan_assy[df_plan_assy['child_uit'].isin(criterios)]

# Reorganizar as colunas na ordem desejada
ordered_columns = ['ID', 'AnotherID', 'org', 'child_item', 'child_uit', 'qpa', 'assy', 'local', 'top_item', 'infor']
filtered_df = filtered_df.reindex(columns=ordered_columns)

# Exibir o DataFrame resultante
filtered_df.head()



,ID,AnotherID,org,child_item,child_uit,qpa,assy,local,top_item,infor
0,NW1AAN30117605,,NW1,AAN30117605,R,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Onhand
1,NW1AAN30117605,AAN3011760532LR650BPSA.AWZSLJZShortage,NW1,AAN30117605,R,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Shortage
4,NW1ACQ30597516,ACQ3059751632LR650BPSA.AWZSLJZShortage,NW1,ACQ30597516,T,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Shortage
5,NW1ACQ30597516,,NW1,ACQ30597516,T,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Onhand
8,NW1AAA31161802,AAA3116180232LR650BPSA.AWZSLJZShortage,NW1,AAA31161802,R,1.0,32LR650BPSA.AWZSLJZ,FA,32LR650BPSA.AWZSLJZ,Shortage


In [62]:
import pandas as pd

# Dados fictícios
data = {
    'org': ['NW1', 'NW1', 'NW1', 'NW1', 'NW1'],
    'child_item': ['AAA30882532', 'AAA30882532', 'AAA30882532', 'AAA30882577', 'AAA30882577'],
    'child_uit': ['R', 'R', 'R', 'R', 'R'],
    'qpa': [1, 1, 1, 1, 1],
    'assy': ['55QNED80TSA.AWZSLSZ', '55QNED80TSA.AWZSLSZ', '55QNED80TSA.AWZSLSZ', '55QNED80TSA.AWZSZSZ', '55QNED80TSA.AWZSZSZ'],
    'local': ['FA', 'FA', 'FA', 'FA', 'FA'],
    'top_item': ['55QNED80TSA/','55QNED80TSA.AWZSLSZ', '55QNED80TSA/', '55QNED80TSA/', '55QNED80TSA/']
}

# Criar o DataFrame diretamente
df_plan_assy = pd.DataFrame(data)

# Criar a coluna ID combinando ORG e CHILD_ITEM
df_plan_assy['ID'] = df_plan_assy['org'].astype(str) + df_plan_assy['child_item'].astype(str)

# Inicializar a coluna 'infor' como vazia
df_plan_assy['infor'] = ''

# Lista para armazenar as novas linhas
new_rows = []

# Preencher a coluna 'infor' com base nas condições
for idx in range(len(df_plan_assy)):
    if df_plan_assy.at[idx, 'local'].lower() == 'FA':
        df_plan_assy.at[idx, 'infor'] = 'Plano'
    else:
        if idx == 0 or df_plan_assy.at[idx, 'child_item'] != df_plan_assy.at[idx - 1, 'child_item']:
            df_plan_assy.at[idx, 'infor'] = 'Onhand'
            # Adiciona 'Shortage' após 'Onhand'
            if idx + 1 < len(df_plan_assy):
                new_row = df_plan_assy.iloc[idx].copy()
                new_row['infor'] = 'Shortage'
                new_rows.append(new_row)
        else:
            df_plan_assy.at[idx, 'infor'] = 'Plano'

# Concatenar as novas linhas ao DataFrame original
df_plan_assy = pd.concat([df_plan_assy] + [pd.DataFrame(new_rows)], ignore_index=True)

# Ordenar o DataFrame por 'org' e 'child_item' para manter a ordem correta
df_plan_assy = df_plan_assy.sort_values(by=['org', 'child_item', 'infor'], ascending=[True, True, False]).reset_index(drop=True)

# Ver os resultados intermediários
print(df_plan_assy[['child_item', 'infor', 'local']])

# Criar a coluna AnotherID com base nas condições
df_plan_assy['AnotherID'] = df_plan_assy.apply(
    lambda row: '' if row['infor'] == 'Onhand' else (
        str(row['child_item']) + str(row['assy']) 
        if row['infor'] == 'Plano' else 
        str(row['child_item']) + str(row['assy']) + 'Shortage'
    ),
    axis=1
)

# Aplicar o filtro na coluna 'child_uit' com os critérios ["M", "P", "R", "T"]
criterios = ["M", "P", "R", "T"]
filtered_df = df_plan_assy[df_plan_assy['child_uit'].isin(criterios)]

# Reorganizar as colunas na ordem desejada
ordered_columns = ['ID', 'AnotherID', 'org', 'child_item', 'child_uit', 'qpa', 'assy', 'local', 'top_item', 'infor']
filtered_df = filtered_df.reindex(columns=ordered_columns)

# Exibir o DataFrame resultante
filtered_df.head()


    child_item     infor local
0  AAA30882532  Shortage    FA
1  AAA30882532     Plano    FA
2  AAA30882532     Plano    FA
3  AAA30882532    Onhand    FA
4  AAA30882577  Shortage    FA
5  AAA30882577     Plano    FA
6  AAA30882577    Onhand    FA


,ID,AnotherID,org,child_item,child_uit,qpa,assy,local,top_item,infor
0,NW1AAA30882532,AAA3088253255QNED80TSA.AWZSLSZShortage,NW1,AAA30882532,R,1,55QNED80TSA.AWZSLSZ,FA,55QNED80TSA/,Shortage
1,NW1AAA30882532,AAA3088253255QNED80TSA.AWZSLSZ,NW1,AAA30882532,R,1,55QNED80TSA.AWZSLSZ,FA,55QNED80TSA.AWZSLSZ,Plano
2,NW1AAA30882532,AAA3088253255QNED80TSA.AWZSLSZ,NW1,AAA30882532,R,1,55QNED80TSA.AWZSLSZ,FA,55QNED80TSA/,Plano
3,NW1AAA30882532,,NW1,AAA30882532,R,1,55QNED80TSA.AWZSLSZ,FA,55QNED80TSA/,Onhand
4,NW1AAA30882577,AAA3088257755QNED80TSA.AWZSZSZShortage,NW1,AAA30882577,R,1,55QNED80TSA.AWZSZSZ,FA,55QNED80TSA/,Shortage
